In [43]:
import urllib
import urllib.robotparser
from dataclasses import dataclass
from bs4 import BeautifulSoup
import re

In [213]:
import pandas as pd

In [17]:
url = "https://www.mhlw.go.jp/stf/seisakunitsuite/bunya/0000177221_00008.html"


In [63]:
@dataclass
class Scraper:
    start_url: str
    target_url: str
    crawl_delay: int = 10
        
    def get_domain(self):
        u = urllib.parse.urlsplit(self.target_url)
        return f"{u.scheme}://{u.hostname}"

    def url_can_fetch(self, robots_file="robots.txt"):
        rp = urllib.robotparser.RobotFileParser()
        # /をつけてjoinするとurlのpath、query stringがなくなる
        rp.set_url(urllib.parse.urljoin(self.target_url, "/", robots_file))
        rp.read()

        if rp.crawl_delay("*") is not None:
            self.crawl_delay = rp.crawl_delay("*")

        return rp.can_fetch("*", self.target_url)

    def request(self, url=None):
        """
        https://qiita.com/hoto17296/items/8fcf55cc6cd823a18217
        """
        if url is None:
            url = self.target_url
        req = urllib.request.Request(url)
        try:
            with urllib.request.urlopen(req) as res:
                body = res.read()

        except urllib.error.HTTPError as e:
            print(e.code, url)
        except urllib.error.URLError as e:
            print(e.reason, url)

        return body

    def parse(self, body=None):
        if body is None:
            body = self.request()
        soup = BeautifulSoup(body, "html.parser")

        # TODO: if exist next url, request next url

        return soup

        

In [64]:
s = Scraper(url, url)
soup = s.parse()


In [410]:
from bs4 import BeautifulSoup, NavigableString, Tag


In [491]:
# 適当にh4のタイトル等つなげて出力
l = []
not_h4 = ['歯科傷病', '歯科傷病', '特定健診', '特定健診（質問票）', '二次医療圏別']

for a in soup.find_all("a", attrs={"href": re.compile(".xlsx$")}):
    #print(a.string)
    h3 = a.find_all_previous("h3")
    h4 = a.find_all_previous("h4")
    h3s = h3[0].string
    h4s = h4[0].string
    #print(h3, h4)
    #print(h3[0].string, h4[0].string, a.string)
    if h3s in not_h4:
        h4s = ""
    if a.string is not None:
        l.append([
            h3s + " " + h4s + " " + a.string,
            s.get_domain() + a.get('href')
        ])


In [493]:
len(l)

153

In [494]:
df = pd.DataFrame(l, columns=['title', 'url'])


In [495]:
df.to_csv('../dataset/urls.csv', index=None)


In [70]:
# urlのみ
a_tags = soup.find_all("a", attrs={"href": re.compile(".xlsx$")})
len(a_tags)
df = pd.DataFrame()
l = []
for a in a_tags:
    l.append([
        a.string,
        s.get_domain() + a.get('href')
    ])

In [74]:
df = pd.DataFrame(l, columns=['title', 'url'])
df.head()

,title,url
0,初再診料_性年齢別算定回数,https://www.mhlw.go.jp/content/12400000/000539...
1,初再診料_都道府県別算定回数,https://www.mhlw.go.jp/content/12400000/000539...
2,短期滞在手術等基本料_性年齢別算定回数,https://www.mhlw.go.jp/content/12400000/000539...
3,短期滞在手術等基本料_都道府県別算定回数,https://www.mhlw.go.jp/content/12400000/000539...
4,特定入院料_性年齢別算定回数,https://www.mhlw.go.jp/content/12400000/000539...


In [75]:
df.to_csv('../urls.csv', index=None)
